In [27]:
import requests
import json

In [28]:
access_token = "*******"

In [29]:
# def get_facebook_pages(access_token):
#     url = "https://graph.facebook.com/v19.0/me/accounts"
#     params = {
#         "access_token": access_token
#     }
#     response = requests.get(url, params=params)
#     return response.json()

# def get_id(access_token, username_required):
#         pages_data = get_facebook_pages(access_token)
#         for page in pages_data['data']:
#                 page_id = page['id']
#                 url = f"https://graph.facebook.com/v19.0/{page_id}"
#                 params = {
#                 "fields": "instagram_business_account",
#                 "access_token": access_token
#                 }
#                 response = requests.get(url, params=params)
#                 if "instagram_business_account" in response.json(): 
#                         ig_business = response.json()
#                         id_ig = ig_business['instagram_business_account']['id']

#                         url = f"https://graph.facebook.com/v19.0/{id_ig}"
#                         params = {
#                                 "fields": ["username"],
#                                 "access_token": access_token
#                         }
#                         response = requests.get(url, params=params)

#                         username = response.json()['username']
#                         if username == username_required:
#                                 return username, id_ig

In [30]:
def get_facebook_pages(access_token):
    url = "https://graph.facebook.com/v19.0/me/accounts"
    params = {
        "access_token": access_token
    }
    response = requests.get(url, params=params)
    return response.json()

def get_id(access_token, usernames_required):
    pages_data = get_facebook_pages(access_token)
    ids = []
    usernames = []
    for page in pages_data['data']:
        page_id = page['id']
        url = f"https://graph.facebook.com/v19.0/{page_id}"
        params = {
            "fields": "instagram_business_account",
            "access_token": access_token
        }
        response = requests.get(url, params=params)
        if "instagram_business_account" in response.json(): 
            ig_business = response.json()
            id_ig = ig_business['instagram_business_account']['id']

            url = f"https://graph.facebook.com/v19.0/{id_ig}"
            params = {
                "fields": "username",
                "access_token": access_token
            }
            response = requests.get(url, params=params)

            username = response.json()['username']
            if username in usernames_required:
                ids.append((username, id_ig))
    
    return ids

In [31]:
id_ig = get_id(access_token, ['gabgalani', 'bodemeier.digital'])
print(id_ig)

[('bodemeier.digital', '17841425444516188'), ('gabgalani', '17841417530400616')]


Função para dados lifetime

In [32]:
# def get_info(access_token, id_ig, fields, output_path, period):
#     url = f"https://graph.facebook.com/v19.0/{id_ig}"
#     params = {
#         "fields": ",".join(fields),
#         "period": period,
#         "access_token": access_token
#     }
    
#     all_data = []
    
#     while url:
#         response = requests.get(url, params=params)
#         data = response.json()
        
#         # Verifique se a resposta contém erros
#         if 'error' in data:
#             print(f"Error: {data['error']['message']}")
#             break
        
#         # Adicione os dados da página atual à lista de todos os dados
#         if 'data' in data:
#             all_data.extend(data['data'])
#         else:
#             all_data.append(data)
        
#         # Verifique se há uma próxima página
#         url = data.get('paging', {}).get('next', None)
        
#         # Após a primeira solicitação, remova os parâmetros para evitar duplicidade
#         params = {}
    
#     # Escreva todos os dados em um arquivo
#     with open(output_path, 'w') as json_file:
#         json.dump(all_data, json_file, indent=4)

In [33]:
def get_info(access_token, ids, fields, output_path, period):
    all_data = []

    for username, id_ig in ids:
        url = f"https://graph.facebook.com/v19.0/{id_ig}"
        params = {
            "fields": ",".join(fields),
            "period": period,
            "access_token": access_token
        }

        while url:
            response = requests.get(url, params=params)
            data = response.json()

            # Verifique se a resposta contém erros
            if 'error' in data:
                print(f"Error fetching data for {username}: {data['error']['message']}")
                break

            # Adicione os dados da página atual à lista de todos os dados
            if 'data' in data:
                for item in data['data']:
                    item['username'] = username  # Adicione o username aos dados
                    all_data.append(item)
            else:
                data['username'] = username  # Adicione o username aos dados
                all_data.append(data)

            # Verifique se há uma próxima página
            url = data.get('paging', {}).get('next', None)

            # Após a primeira solicitação, remova os parâmetros para evitar duplicidade
            params = {}

    # Escreva todos os dados em um arquivo
    with open(output_path, 'w') as json_file:
        json.dump(all_data, json_file, indent=4)

## Coletando cabeçalho de 3 em 3 dias 

In [34]:
# Roda de 3 em 3 dias

fields = ['id','username','name','biography', 'followers_count','follows_count', 'profile_picture_url', 'media_count']
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\tbAccount_ig.json'
get_info(access_token, id_ig, fields, output_path, 'lifetime')

## Coletando medias sem insights(Rodar a primeira vez geral, depois rodar 1 vez ao dia, e uma vez na semana rodar todo periodo)

In [35]:
# fields=
# media{comments_count,like_count, media_type, media_url, caption, timestamp, permalink, media_product_type, thumbnail_url, shortcode, children{media_type, media_url, id}, owner{id, username}, comments{username, text, timestamp}, insights.metric(reach, impressions, engagement)}

In [36]:
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TbMedias.json'
fields_no_insights = [
    'id','media{id, ad_id, comments_count,like_count, media_type, media_url, caption, timestamp, permalink, media_product_type, thumbnail_url, shortcode, children{media_type, media_url}, owner{id, username}}'
]
get_info(access_token, id_ig, fields_no_insights, output_path, 'lifetime')

## Coletando os dados dos stroires sem insigts (rodar diáriamente)

In [37]:
# 17841417530400616?fields=
# stories{comments_count,like_count, media_type, media_url, caption, timestamp, permalink, media_product_type, thumbnail_url, shortcode, children{media_type, media_url, id}, owner{id, username}, insights.metric(reach, impressions, engagement)}
# 17841417530400616?fields=
# stories{comments_count,like_count, media_type, media_url, caption, timestamp, permalink, media_product_type, thumbnail_url, shortcode, children{media_type, media_url, id}, owner{id, username}}

In [38]:

fields_story_no_in = [
    'id, stories{comments_count,like_count, media_type, media_url, caption, timestamp, permalink, media_product_type, thumbnail_url, shortcode, children{media_type, media_url, id}, owner{id, username}}'
]
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TbStories.json'
get_info(access_token, id_ig, fields_story_no_in, output_path, 'day')

# Dados da conta com insights

## Day level metrics (Baixar giariamente)

In [39]:
# impressions, reach, follower_count, email_contacts, phone_call_clicks, text_message_clicks, get_directions_clicks, website_clicks, profile_views

In [40]:
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TbDayAccounIgMetrics.json'
day_level_metrics = [
  'id, insights.metric(impressions, reach, follower_count,email_contacts, phone_call_clicks, text_message_clicks,get_directions_clicks,website_clicks,profile_views).period(day)'
]
get_info(access_token, id_ig, day_level_metrics, output_path, 'day')

## Lifetime level metrics  (diariamente)

In [41]:
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TbLifetimeAccounIgMetrics.json'
lifetime_level_metrics = [
'id, insights.metric(audience_gender_age, audience_locale, audience_country, audience_city, online_followers).period(lifetime)'
]
get_info(access_token, id_ig, lifetime_level_metrics, output_path, 'lifetime')

# Dados da midia com Insights

In [42]:
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TbMediaInsightsLifetime.json'
lifetime_level_metrics = [
'id, media{id, insights.metric(impressions, reach, taps_forward, taps_back, exits, replies, engagement, saved, video_views)}']
get_info(access_token, id_ig, lifetime_level_metrics, output_path, 'lifetime')

Error fetching data for gabgalani: Invalid parameter


## Dados dos stories com insghts

In [43]:
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TbStoriesInsightsLifetime.json'
lifetime_level_metrics_stories = [
'id, stories{id, insights.metric(exits, impressions, reach, replies, taps_forward, taps_back)}']
get_info(access_token, id_ig, lifetime_level_metrics_stories, output_path, 'lifetime')

## ADS

In [44]:
output_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\TESTE_ADS.json'
lifetime_level_metrics_stories = [
'ads_posts{actions}'
]
get_info(token_ads, id_ig, lifetime_level_metrics_stories, output_path, 'lifetime')

Error fetching data for bodemeier.digital: (#100) Tried accessing nonexisting field (ads_posts) on node type (IGUser)
Error fetching data for gabgalani: (#100) Tried accessing nonexisting field (ads_posts) on node type (IGUser)


chamada adsset

In [45]:
# act_124317524325975/adsets?fields=id,name, status,promoted_object, daily_budget, lifetime_budget, start_time, end_time, targeting, bid_strategy, optimization_goal, billing_event